In [ ]:
!pip install ultralytics

In [2]:
import os
import shutil
import random

In [ ]:
!pip install tqdm --upgrade
from tqdm.notebook import tqdm

In [ ]:
from ultralytics import YOLO
ultralytics.checks()

In [5]:
DRIVE_BASE_PATH = "/content/drive/MyDrive/shuffle_train_invert_clean_new/"
TRAIN_DRIVE_BASE_PATH =  os.path.join(DRIVE_BASE_PATH, "train/")  # DRIVE_BASE_PATH переехало сюда!!!
SOURCE_DATA_PATH = os.path.join(DRIVE_BASE_PATH, "chest_xrays")  # Folder with images and labels
print(TRAIN_DRIVE_BASE_PATH)

/content/drive/MyDrive/shuffle_train_invert_clean_new/train/


In [ ]:
train_path_img = "./yolo_data/images/train/"
train_path_label = "./yolo_data/labels/train/"
val_path_img = "./yolo_data/images/val/"
val_path_label = "./yolo_data/labels/val/"
test_path = "./yolo_data/test"

In [ ]:
'''
Split the dataset into train and test and creates the train.txt and test.tx with
the respective path of the images in each folder
'''

def train_test_split(path, neg_path=None, split=0.2):
    print("------ PROCESS STARTED -------")

    files = list(set([name[:-4] for name in os.listdir(path)]))
    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files) * split)

    # РАЗДЕЛЕНИЕ на непересекающиеся части
    train_files = files[:-test_size]  # 80% для тренировки
    val_files = files[-test_size:]    # 20% для валидации

    # Создание директорий
    os.makedirs(train_path_img, exist_ok=True)
    os.makedirs(train_path_label, exist_ok=True)
    os.makedirs(val_path_img, exist_ok=True)
    os.makedirs(val_path_label, exist_ok=True)

    # Копирование ТОЛЬКО тренировочных данных
    for filex in tqdm(train_files):
        if filex == 'classes':
            continue
        shutil.copy2(path + filex + '.jpg', f"{train_path_img}/" + filex + '.jpg')
        shutil.copy2(path + filex + '.txt', f"{train_path_label}/" + filex + '.txt')

    print(f"------ Training data created with {len(train_files)} images -------")

    # Копирование ТОЛЬКО валидационных данных
    for filex in tqdm(val_files):
        if filex == 'classes':
            continue
        shutil.copy2(path + filex + '.jpg', f"{val_path_img}/" + filex + '.jpg')
        shutil.copy2(path + filex + '.txt', f"{val_path_label}/" + filex + '.txt')

    print(f"------ Validation data created with {len(val_files)} images ----------")
    print("------ TASK COMPLETED -------")

## spliting the data into train-test and creating train.txt and test.txt files
# train_test_split('/content/drive/MyDrive/custom_notebooks/yolo_data/')

### for label_tag
train_test_split(TRAIN_DRIVE_BASE_PATH) ### without negative images
# train_test_split('./data/','./negative_images/') ### if you want to feed negative images

In [ ]:
model = YOLO('yolov8s.pt')

results = model.train(
    data='/content/drive/MyDrive/shuffle_train_invert_clean_new/dataset_task3.yaml',
    epochs=200,
    imgsz=640,
    batch=8,
    project='/content/drive/MyDrive/shuffle_train_invert_clean_new/yolo_training_results',
    name='foreign_items_task10',
    exist_ok=True
)

In [ ]:
model = YOLO('/content/drive/MyDrive/shuffle_train_invert_clean_new/yolo_training_results/foreign_items_task10/weights/best.pt')

results = model.predict(
    source='/content/drive/MyDrive/shuffle_test',
    conf=0.25,
    save_txt=True
)